Friday US SP500 Movements vs Monday SP500 ETF Movement in JPY

In [3]:
from tiportfolio.helpers.data import Alpaca
import dotenv

dotenv.load_dotenv()

import os
alpaca = Alpaca(os.environ['ALPACA_API_KEY'], os.environ['ALPACA_API_SECRET'])

In [4]:
# spy
spy_df = alpaca.query('SPY', '2010-01-01', '2024-01-01', '1D', True)
spy_df

Loading bar data...


ValueError: Unknown adjustment: True.

In [7]:
from tiportfolio.helpers.data import YFinance

yfinance = YFinance()
df = yfinance.query(['AAPL', 'MSFT'], start_date='3/1/2021', end_date='3/1/2022')
df


Loading bar data...


ERROR:yfinance:Failed to get ticker 'MSFT' reason: Failed to perform, curl: (60) SSL certificate problem: certificate has expired. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
ERROR:yfinance:Failed to get ticker 'AAPL' reason: Failed to perform, curl: (60) SSL certificate problem: certificate has expired. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  2 of 2 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['MSFT', 'AAPL']: YFTzMissingError('possibly delisted; no timezone found')


ValueError: DataFrame is missing required columns: ['symbol', 'date', 'open', 'high', 'low', 'close']

In [6]:
# get 2558.T
etf_jpy_df = yfinance.query('AAPL', '2010-01-01', '2024-01-01', '1D', True)
etf_jpy_df

Loading bar data...


ERROR:yfinance:Failed to get ticker 'AAPL' reason: Failed to perform, curl: (60) SSL certificate problem: certificate has expired. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFTzMissingError('possibly delisted; no timezone found')


ValueError: DataFrame is missing required columns: ['symbol', 'date', 'open', 'high', 'low', 'close']